In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import sys
from tqdm import tqdm
from pathlib import Path
import tensorflow as tf
import skimage.io
import matplotlib.pyplot as plt

from skimage.color import rgb2gray
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"Thresholding is a type of image segmentation, where we change the pixels of an image to make the image easier to analyze. In thresholding, we convert an image from color or grayscale into a binary image, i.e., one that is simply black and white."

https://datacarpentry.org/image-processing/07-thresholding/#:~:text=Thresholding%20is%20a%20type%20of,is%20simply%20black%20and%20white.

![](https://slidetodoc.com/presentation_image_h/7e87127686d09dcae37cf586e988e6b9/image-33.jpg)SlideToDoc.com

#Prepare mask_rcnn

Spoiler alert: I couldn't perform the masks images. 

In [ ]:
# https://www.kaggle.com/pednoi/training-mask-r-cnn-to-be-a-fashionista-lb-0-07

#Code by S_Shohei https://www.kaggle.com/code/iiyamaiiyama/how-to-submit-prediction

!git clone https://www.github.com/matterport/Mask_RCNN.git
os.chdir('Mask_RCNN')

!rm -rf .git # to prevent an error when the kernel is committed
!rm -rf images assets # to prevent displaying images at the bottom of a kernel

In [ ]:
#Code by S_Shohei https://www.kaggle.com/code/iiyamaiiyama/how-to-submit-prediction

DATA_DIR = Path('/kaggle/input')
ROOT_DIR = Path('/kaggle/working')

In [ ]:
#Code by S_Shohei https://www.kaggle.com/code/iiyamaiiyama/how-to-submit-prediction

sys.path.append(ROOT_DIR/'Mask_RCNN')

In [ ]:
!pip install pycocotools

In [ ]:
#Code by S_Shohei https://www.kaggle.com/code/iiyamaiiyama/how-to-submit-prediction

from mrcnn.config import Config
from mrcnn import utils
#import mrcnn.model as modellib
from mrcnn import visualize
#from mrcnn.model import log
#import keras.engine.topology as KE
from tensorflow.keras.layers import Layer

In [ ]:
!wget https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5

In [ ]:
COCO_MODEL_PATH = 'mask_rcnn_coco.h5'

#StackOverflow trying to fix AttributeError: module 'keras.engine' has no attribute 'Layer'

In [ ]:
import tensorflow
tensorflow.__version__

In [ ]:
!pip uninstall keras -y
!pip uninstall keras-nightly -y
!pip uninstall keras-Preprocessing -y
!pip uninstall keras-vis -y
!pip uninstall tensorflow -y
!pip uninstall h5py -y

In [ ]:
!pip install tensorflow==1.13.1
!pip install keras==2.0.8
!pip install h5py==2.10.0

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras.layers as KE

import keras.engine.topology as KE

In [ ]:
#from tensorflow.keras.layers import Layer

# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "Mask_RCNN/samples/coco/"))  # To find local version
import coco

#After AttributeError: module 'keras.engine' has no attribute 'Layer'

Many answers on StackOverflow that doesn't give any result.

I wasn't able to proceed with Masks RCNN 

In [ ]:
#Code by Volody  https://www.kaggle.com/code/volody/threshold-segmentation

img_list = ['000011451.jpg', '000011444.jpg', '000011447.jpg', '000011456.jpg']
fig = plt.figure(figsize=(16, 16))
for i in range(4):
    x = fig.add_subplot(2, 2, i+1)
    image = plt.imread('/kaggle/input/hotel-id-to-combat-human-trafficking-2022-fgvc9/train_images/10290/'+img_list[i])
    x.set_title("{image} ({shape[0]},{shape[1]})".format(image=img_list[i],shape=image.shape))
    plt.imshow(image)

#Convert it into grayscale, that allows using simple treshold processing

In [ ]:
#Code by Volody  https://www.kaggle.com/code/volody/threshold-segmentation

fig = plt.figure(figsize=(16, 16))
for i in range(4):
    x = fig.add_subplot(2, 2, i+1)
    image = plt.imread('/kaggle/input/hotel-id-to-combat-human-trafficking-2022-fgvc9/train_images/10290/'+img_list[i])
    gray = rgb2gray(image)
    x.set_title("{image} ({shape[0]},{shape[1]})".format(image=img_list[i],shape=gray.shape))
    plt.imshow(gray, cmap='gray')

In [ ]:
#Code by Volody  https://www.kaggle.com/code/volody/threshold-segmentation

def to_2regions(image):
    gray = rgb2gray(image)
    m = gray.mean()
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):    
            gray[i,j] = int(gray[i,j] > m)
    return gray

In [ ]:
#Code by Volody  https://www.kaggle.com/code/volody/threshold-segmentation

fig = plt.figure(figsize=(16, 16))
for i in range(4):
    x = fig.add_subplot(2, 2, i+1)
    image = plt.imread('/kaggle/input/hotel-id-to-combat-human-trafficking-2022-fgvc9/train_images/10290/'+img_list[i])
    r2_gray = to_2regions(image)
    x.set_title("{image} ({shape[0]},{shape[1]})".format(image=img_list[i],shape=r2_gray.shape))
    plt.imshow(r2_gray, cmap='gray')

#Thresholds to detect multiple objects:

In [ ]:
#Code by Volody  https://www.kaggle.com/code/volody/threshold-segmentation

def to_4regions(image):
    g_image = rgb2gray(image)
    m = g_image.mean()
    m1 = 0.5 * m
    m2 = 0.25 * m
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            pxl = g_image[i,j]
            if pxl > m:
                g_image[i,j] = 3
            elif pxl > m1:
                g_image[i,j] = 2
            elif pxl > m2:
                g_image[i,j] = 1
            else:
                g_image[i,j] = 0            
    return g_image

In [ ]:
#Code by Volody  https://www.kaggle.com/code/volody/threshold-segmentation

fig = plt.figure(figsize=(16, 16))
for i in range(4):
    x = fig.add_subplot(2, 2, i+1)
    image = plt.imread('/kaggle/input/hotel-id-to-combat-human-trafficking-2022-fgvc9/train_images/10290/'+img_list[i])
    r2_gray = to_4regions(image)
    x.set_title("{image} ({shape[0]},{shape[1]})".format(image=img_list[i],shape=r2_gray.shape))
    plt.imshow(r2_gray, cmap='gray')

Unfortunately I didn't work with the masks, yet.

#Acknowledgements:

Code by Volody  https://www.kaggle.com/code/volody/threshold-segmentation

S_Shohei https://www.kaggle.com/code/iiyamaiiyama/how-to-submit-prediction